In [2]:
#import needed libraries
import gensim
from gensim.models import KeyedVectors
from gensim.models import FastText
import numpy as np
from numpy import *
import scipy
from scipy.spatial import distance

In [3]:
#set a function to turn the text into list with every sentence that tkoenized into words as sublists
def getIndex2word(filename):
    index2word = []
    word2index = []
    with open(filename, encoding='utf-8') as fr:
        for line in fr:
            tmp = {}
            newline = line.strip().split()
            index2word.append({index:word for index, word in enumerate(newline, 0)})
            word2index.append({word:index for index, word in enumerate(newline, 0)})
    return index2word, word2index

In [5]:
#read the English and Chinese text file
zh_index2word, zh_word2index = getIndex2word("中文.txt")
en_index2word, en_word2index = getIndex2word("英文.txt")

In [6]:
#read the results of word alignment, and turn it into a list with every line that tkoenized as sublists
def getCDQ():
    datas = []
    with open('词对齐.txt', encoding='utf-8') as fr:
        for line in fr:
            datas.append(line.strip().split())
    return datas
datas = getCDQ()

In [7]:
#load the bilingual mapped Chinese model
ZH_model = KeyedVectors.load_word2vec_format("TRG_HT_MAPPED.EMB", binary=False, unicode_errors="jgnore")

In [8]:
#load the bilingual mapped English model
EN_model = KeyedVectors.load_word2vec_format("SRC_HT_MAPPED.EMB", binary=False, unicode_errors="jgnore")

In [9]:
#set a function to query vector in English model
def get_en_vector(word):
    vector_en = EN_model.wv.get_vector(word)
    return vector_en

In [10]:
#set a function to query vector in Chinese model
def get_zh_vector(word):
    vector_zh = ZH_model.wv.get_vector(word)
    return vector_zh

In [12]:
#get the cosine distance of word pairs
for index, (d, zh, en) in enumerate(zip(datas, zh_index2word, en_index2word)): 
    dists = []
    skipped = []
    for a in d:
        en_wid, zh_wid = a.split("-")
        en_word = en[int(en_wid)]
        zh_word = zh[int(zh_wid)]
#         print(en_wid, en_word, zh_wid, zh_word)
        if en_word in EN_model.wv.vocab and zh_word in ZH_model.wv.vocab:
            w_en_vector = get_en_vector(en_word)
            #print("en vector:", w_en_vector)
            w_zh_vector = get_zh_vector(zh_word)
            #print("zh vector:", w_zh_vector)
            distance = (scipy.spatial.distance.cosine(w_en_vector, w_zh_vector))
            #print("distance:", distance)
            dists.append(distance)
        else:
            skipped.append(a)
print(dists)

[1.2307753413915634, 0.715864360332489, 0.8030718713998795, 0.8467187583446503, 1.158906102180481, 0.9721608012914658, 0.8436356484889984, 0.8065491765737534, 1.1719548106193542, 0.8199136853218079, 0.8982102572917938, 0.8401868790388107]


<ipython-input-12-8a3a1addb479>:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if en_word in EN_model.wv.vocab and zh_word in ZH_model.wv.vocab:
<ipython-input-9-9ffdebc57d0f>:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vector_en = EN_model.wv.get_vector(word)


In [13]:
#automatically select the Source words whose cosine distance of its word pairs are in the setting range (irrational range)
def getTarget(alph1, alph2):
    Auto_UCP_MT = []
    for index, (d, zh, en) in enumerate(zip(datas, zh_index2word, en_index2word)): 
        dists = []
        for a in d:
            en_wid, zh_wid = a.split("-")
            en_word = en[int(en_wid)]
            zh_word = zh[int(zh_wid)]
#             print(en_wid, en_word, zh_wid, zh_word)
            if en_word in EN_model.wv.vocab and zh_word in ZH_model.wv.vocab:
                w_en_vector = get_en_vector(en_word)
                #print("en vector:", w_en_vector)
                w_zh_vector = get_zh_vector(zh_word)
                #print("zh vector:", w_zh_vector)
                distance = (scipy.spatial.distance.cosine(w_en_vector, w_zh_vector))
                #print("distance:", distance)
                if distance>=alph1 and distance<=alph2:
                    dists.append(en_word)  
            else:
                pass
        Auto_UCP_MT.append(dists)
        print(dists)
    return Auto_UCP_MT

In [14]:
#get the words within the setting range per line 
Auto_UCP_MT = getTarget(0.8, 1)

['pages', 'pages', 'former', 'wore']
['eccentricities', 'his', 'claimed', 'deliberate', 'his', 'former', 'media', 'late', 'cultivated']
['Metro', '.', 'persona', 'told', 'stop']
['spark', 'he', 'conversation', '.', 'put', 'surgical', 'pages']
['form', 'you', 'your', 'true', 'what', 'tell', '.', 'own']


<ipython-input-13-2edcdd8b3fcb>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if en_word in EN_model.wv.vocab and zh_word in ZH_model.wv.vocab:
<ipython-input-9-9ffdebc57d0f>:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vector_en = EN_model.wv.get_vector(word)


In [25]:
# wrtie every line of words in a text; the line is aligns to that in ST 
with open('写入.txt', 'w') as output_num:
    for sublist in Auto_UCP_MT:
        for i in sublist:
            output_num.write(str(i))
            output_num.write(' ')
        output_num.write('\n')

In [16]:
#Below are codes for another function: if I have got the word order of units of creative potential in ST, I can use these word order to get their word pair in TT
#first I low the txt containing the word order of units of creative potential in ST
targets = []
with open('数字.txt', encoding='utf-8') as fr:
    for line in fr:
        targets.append(line.strip().split(','))
targets

[['8', '9'], ['0', '1'], ['1', '2'], ['5'], ['2']]

In [19]:
#read the file with word alignment results
datas[:3]

[['3-14',
  '0-4',
  '10-8',
  '1-5',
  '10-14',
  '13-1',
  '12-0',
  '0-3',
  '11-2',
  '7-6',
  '2-12',
  '6-10',
  '9-8',
  '9-14'],
 ['16-20',
  '18-20',
  '14-16',
  '9-11',
  '1-3',
  '19-21',
  '13-14',
  '6-8',
  '5-6',
  '22-22',
  '0-1',
  '17-17',
  '2-4',
  '1-2',
  '13-15',
  '3-5',
  '23-23',
  '0-0',
  '9-12',
  '8-10',
  '10-13',
  '16-14',
  '7-8'],
 ['12-4',
  '4-0',
  '18-17',
  '24-36',
  '5-1',
  '40-29',
  '34-31',
  '23-40',
  '13-5',
  '19-21',
  '1-12',
  '6-2',
  '32-27',
  '41-30',
  '3-0',
  '14-6',
  '29-25',
  '42-44',
  '25-37',
  '20-22',
  '26-38',
  '17-14',
  '11-4',
  '0-10',
  '36-33',
  '19-20',
  '30-26',
  '15-7',
  '7-3',
  '33-28',
  '27-42',
  '18-18',
  '18-15',
  '38-30',
  '21-23',
  '31-27',
  '22-39',
  '2-9',
  '1-13',
  '28-43']]

In [18]:
#set an empty list for word pairs of units of creative potential
UCP = []

In [20]:
#query the aligned word pair the word order, and turn the word pair in a list per line
for tg, das in zip(targets, datas):
    tmp = []
    for l in tg:
        for k in das:
            kx = k.split('-')
            if l == kx[0]:
                tmp.append(k)
    print(tmp)
    UCP.append(tmp)

['9-8', '9-14']
['0-1', '0-0', '1-3', '1-2']
['1-12', '1-13', '2-9']
['5-7']
['2-3']
